# Unit 5: Univariate Analysis - Practical Exercises

## Table of Contents
1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Analyzing Numerical Variables](#analyzing-numerical)
4. [Measures of Central Tendency](#central-tendency)
5. [Measures of Dispersion](#dispersion)
6. [Measures of Shape](#shape)
7. [Visualizing Numerical Variables](#visualizing-numerical)
8. [Analyzing Categorical Variables](#analyzing-categorical)
9. [Visualizing Categorical Variables](#visualizing-categorical)
10. [Distribution Analysis](#distribution-analysis)
11. [Complete Analysis Workflows](#complete-workflows)

---

## 1. Introduction <a id='introduction'></a>

In this practical notebook, you'll learn to perform comprehensive univariate analysis on both numerical and categorical variables. You'll master descriptive statistics, create insightful visualizations, and interpret distributions.

**Learning Objectives:**
- Calculate and interpret measures of central tendency and dispersion
- Analyze distribution shape (skewness, kurtosis)
- Create effective visualizations for numerical data
- Analyze and visualize categorical variables
- Test for normality and understand distribution types
- Build complete analysis workflows

## 2. Setup <a id='setup'></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import skew, kurtosis, shapiro, norm

# Settings
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
np.random.seed(42)

print("✅ Setup complete!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {scipy.__version__}")

## 3. Analyzing Numerical Variables <a id='analyzing-numerical'></a>

Let's start with a comprehensive analysis of numerical variables.

In [ ]:
# Load datasets
tips = sns.load_dataset('tips')
titanic = sns.load_dataset('titanic')

print("DATASETS LOADED")
print("="*80)
print(f"Tips dataset: {tips.shape}")
print(f"Titanic dataset: {titanic.shape}")

# Display sample
print("\nTips dataset sample:")
print(tips.head())

## 4. Measures of Central Tendency <a id='central-tendency'></a>

Let's calculate mean, median, and mode for numerical variables.

In [ ]:
# Analyze 'total_bill' from tips dataset
print("CENTRAL TENDENCY ANALYSIS: Total Bill")
print("="*80)

mean_bill = tips['total_bill'].mean()
median_bill = tips['total_bill'].median()
mode_bill = tips['total_bill'].mode()

print(f"Mean:   ${mean_bill:.2f}")
print(f"Median: ${median_bill:.2f}")
print(f"Mode:   ${mode_bill.values[0] if len(mode_bill) > 0 else 'No unique mode':.2f}")

# Interpretation
diff = abs(mean_bill - median_bill)
print(f"\nMean - Median = ${diff:.2f}")

if diff < 1:
    print("✅ Distribution is approximately symmetric (mean ≈ median)")
elif mean_bill > median_bill:
    print("⚠️ Distribution is right-skewed (mean > median)")
    print("   → High values pulling the mean up")
else:
    print("⚠️ Distribution is left-skewed (mean < median)")
    print("   → Low values pulling the mean down")

In [ ]:
# Visualize central tendency
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with mean and median
axes[0].hist(tips['total_bill'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(mean_bill, color='red', linestyle='--', linewidth=2, 
                label=f'Mean: ${mean_bill:.2f}')
axes[0].axvline(median_bill, color='green', linestyle='--', linewidth=2, 
                label=f'Median: ${median_bill:.2f}')
axes[0].set_xlabel('Total Bill ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Total Bill with Central Tendency', fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Box plot
box_data = axes[1].boxplot(tips['total_bill'], vert=True, patch_artist=True)
for patch in box_data['boxes']:
    patch.set_facecolor('lightblue')
axes[1].set_ylabel('Total Bill ($)')
axes[1].set_title('Box Plot of Total Bill', fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Add median line annotation
axes[1].text(1.15, median_bill, f'Median: ${median_bill:.2f}', 
             verticalalignment='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Compare mean and median for different scenarios
print("\nCOMPARING CENTRAL TENDENCY ACROSS VARIABLES")
print("="*80)

variables = ['total_bill', 'tip']

for var in variables:
    mean_val = tips[var].mean()
    median_val = tips[var].median()
    
    print(f"\n{var.upper()}:")
    print(f"  Mean:   ${mean_val:.2f}")
    print(f"  Median: ${median_val:.2f}")
    print(f"  Ratio (Mean/Median): {mean_val/median_val:.3f}")
    
    if mean_val / median_val > 1.1:
        print("  → Right-skewed")
    elif mean_val / median_val < 0.9:
        print("  → Left-skewed")
    else:
        print("  → Approximately symmetric")

## 5. Measures of Dispersion <a id='dispersion'></a>

Now let's analyze how spread out the data is.

In [ ]:
# Calculate dispersion measures
print("DISPERSION ANALYSIS: Total Bill")
print("="*80)

# Range
data_min = tips['total_bill'].min()
data_max = tips['total_bill'].max()
data_range = data_max - data_min

print(f"Minimum: ${data_min:.2f}")
print(f"Maximum: ${data_max:.2f}")
print(f"Range:   ${data_range:.2f}")

# Quartiles and IQR
Q1 = tips['total_bill'].quantile(0.25)
Q2 = tips['total_bill'].quantile(0.50)  # Median
Q3 = tips['total_bill'].quantile(0.75)
IQR = Q3 - Q1

print(f"\nQuartiles:")
print(f"  Q1 (25th percentile): ${Q1:.2f}")
print(f"  Q2 (50th percentile): ${Q2:.2f}")
print(f"  Q3 (75th percentile): ${Q3:.2f}")
print(f"  IQR (Q3 - Q1):        ${IQR:.2f}")

# Variance and Standard Deviation
variance = tips['total_bill'].var()
std_dev = tips['total_bill'].std()

print(f"\nVariance:          ${variance:.2f}")
print(f"Standard Deviation: ${std_dev:.2f}")

# Coefficient of Variation
cv = (std_dev / mean_bill) * 100
print(f"Coefficient of Variation: {cv:.1f}%")

In [ ]:
# Visualize dispersion
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram with std dev bands
axes[0, 0].hist(tips['total_bill'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].axvline(mean_bill, color='red', linestyle='--', linewidth=2, label='Mean')
axes[0, 0].axvline(mean_bill - std_dev, color='orange', linestyle=':', linewidth=2, label='±1 SD')
axes[0, 0].axvline(mean_bill + std_dev, color='orange', linestyle=':', linewidth=2)
axes[0, 0].axvline(mean_bill - 2*std_dev, color='green', linestyle=':', linewidth=1, label='±2 SD')
axes[0, 0].axvline(mean_bill + 2*std_dev, color='green', linestyle=':', linewidth=1)
axes[0, 0].set_xlabel('Total Bill ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution with Standard Deviation Bands', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# Box plot showing quartiles
bp = axes[0, 1].boxplot(tips['total_bill'], vert=True, patch_artist=True,
                         boxprops=dict(facecolor='lightblue'))
axes[0, 1].set_ylabel('Total Bill ($)')
axes[0, 1].set_title('Box Plot Showing IQR', fontweight='bold')
axes[0, 1].text(1.3, Q1, f'Q1: ${Q1:.2f}', verticalalignment='center')
axes[0, 1].text(1.3, Q2, f'Q2: ${Q2:.2f}', verticalalignment='center')
axes[0, 1].text(1.3, Q3, f'Q3: ${Q3:.2f}', verticalalignment='center')
axes[0, 1].grid(axis='y', alpha=0.3)

# Percentile plot
percentiles = [np.percentile(tips['total_bill'], i) for i in range(0, 101)]
axes[1, 0].plot(range(0, 101), percentiles, linewidth=2)
axes[1, 0].axhline(Q1, color='red', linestyle='--', alpha=0.5, label='Q1')
axes[1, 0].axhline(Q2, color='green', linestyle='--', alpha=0.5, label='Q2')
axes[1, 0].axhline(Q3, color='blue', linestyle='--', alpha=0.5, label='Q3')
axes[1, 0].set_xlabel('Percentile')
axes[1, 0].set_ylabel('Total Bill ($)')
axes[1, 0].set_title('Percentile Plot', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Range comparison
data_comparison = {
    'Total Bill': [tips['total_bill'].min(), tips['total_bill'].max(), tips['total_bill'].std()],
    'Tip': [tips['tip'].min(), tips['tip'].max(), tips['tip'].std()]
}
x_pos = np.arange(len(data_comparison))
axes[1, 1].bar(x_pos, [data_comparison[k][1] - data_comparison[k][0] for k in data_comparison.keys()],
               alpha=0.7, edgecolor='black')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(data_comparison.keys())
axes[1, 1].set_ylabel('Range ($)')
axes[1, 1].set_title('Range Comparison', fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Check how much data falls within ±1, ±2, ±3 SD (for normal distribution)
print("\nEMPIRICAL RULE CHECK (68-95-99.7 Rule)")
print("="*80)

within_1sd = ((tips['total_bill'] >= mean_bill - std_dev) & 
              (tips['total_bill'] <= mean_bill + std_dev)).sum()
within_2sd = ((tips['total_bill'] >= mean_bill - 2*std_dev) & 
              (tips['total_bill'] <= mean_bill + 2*std_dev)).sum()
within_3sd = ((tips['total_bill'] >= mean_bill - 3*std_dev) & 
              (tips['total_bill'] <= mean_bill + 3*std_dev)).sum()

total = len(tips)

print(f"Within ±1 SD: {within_1sd}/{total} ({within_1sd/total*100:.1f}%) - Expected ~68%")
print(f"Within ±2 SD: {within_2sd}/{total} ({within_2sd/total*100:.1f}%) - Expected ~95%")
print(f"Within ±3 SD: {within_3sd}/{total} ({within_3sd/total*100:.1f}%) - Expected ~99.7%")

if abs(within_1sd/total*100 - 68) < 10:
    print("\n✅ Data roughly follows normal distribution pattern")
else:
    print("\n⚠️ Data doesn't follow normal distribution pattern")

## 6. Measures of Shape <a id='shape'></a>

Analyze distribution shape using skewness and kurtosis.

In [ ]:
# Calculate shape measures
print("SHAPE ANALYSIS: Total Bill")
print("="*80)

skewness = skew(tips['total_bill'])
kurt = kurtosis(tips['total_bill'])

print(f"Skewness: {skewness:.3f}")
if abs(skewness) < 0.5:
    skew_interpretation = "Approximately symmetric"
elif skewness > 0:
    skew_interpretation = "Right-skewed (positive skew)"
else:
    skew_interpretation = "Left-skewed (negative skew)"
print(f"  → {skew_interpretation}")

print(f"\nKurtosis (excess): {kurt:.3f}")
if abs(kurt) < 0.5:
    kurt_interpretation = "Normal (mesokurtic)"
elif kurt > 0:
    kurt_interpretation = "Heavy-tailed (leptokurtic) - more outliers"
else:
    kurt_interpretation = "Light-tailed (platykurtic) - fewer outliers"
print(f"  → {kurt_interpretation}")

In [ ]:
# Visualize skewness with examples
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Right-skewed data (like total_bill)
axes[0].hist(tips['total_bill'], bins=30, edgecolor='black', alpha=0.7, color='coral')
axes[0].axvline(tips['total_bill'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0].axvline(tips['total_bill'].median(), color='green', linestyle='--', linewidth=2, label='Median')
axes[0].set_xlabel('Total Bill ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title(f'Right-Skewed Data\nSkewness: {skew(tips["total_bill"]):.2f}', fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Approximately symmetric data
symmetric_data = np.random.normal(50, 10, 1000)
axes[1].hist(symmetric_data, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[1].axvline(symmetric_data.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[1].axvline(np.median(symmetric_data), color='green', linestyle='--', linewidth=2, label='Median')
axes[1].set_xlabel('Value')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'Symmetric Data\nSkewness: {skew(symmetric_data):.2f}', fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# Left-skewed data
left_skewed_data = 100 - np.random.exponential(15, 1000)
axes[2].hist(left_skewed_data, bins=30, edgecolor='black', alpha=0.7, color='lightgreen')
axes[2].axvline(left_skewed_data.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[2].axvline(np.median(left_skewed_data), color='green', linestyle='--', linewidth=2, label='Median')
axes[2].set_xlabel('Value')
axes[2].set_ylabel('Frequency')
axes[2].set_title(f'Left-Skewed Data\nSkewness: {skew(left_skewed_data):.2f}', fontweight='bold')
axes[2].legend()
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("INTERPRETATION:")
print("="*80)
print("Right-skewed: Mean > Median (tail extends to the right)")
print("Symmetric:    Mean ≈ Median (balanced distribution)")
print("Left-skewed:  Mean < Median (tail extends to the left)")

## 7. Visualizing Numerical Variables <a id='visualizing-numerical'></a>

Create comprehensive visualizations for numerical data.

In [ ]:
# Create multiple visualization types
print("COMPREHENSIVE VISUALIZATION: Total Bill")
print("="*80)

fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Histogram
ax1 = fig.add_subplot(gs[0, :2])
ax1.hist(tips['total_bill'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(mean_bill, color='red', linestyle='--', linewidth=2, label=f'Mean: ${mean_bill:.2f}')
ax1.axvline(median_bill, color='green', linestyle='--', linewidth=2, label=f'Median: ${median_bill:.2f}')
ax1.set_xlabel('Total Bill ($)', fontsize=11)
ax1.set_ylabel('Frequency', fontsize=11)
ax1.set_title('Histogram', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 2. Box Plot
ax2 = fig.add_subplot(gs[0, 2])
bp = ax2.boxplot(tips['total_bill'], vert=True, patch_artist=True,
                  boxprops=dict(facecolor='lightblue', alpha=0.7))
ax2.set_ylabel('Total Bill ($)', fontsize=11)
ax2.set_title('Box Plot', fontsize=12, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# 3. Density Plot (KDE)
ax3 = fig.add_subplot(gs[1, :2])
tips['total_bill'].plot(kind='density', ax=ax3, linewidth=2, color='darkblue')
ax3.fill_between(ax3.get_xlim(), 0, tips['total_bill'].plot(kind='density').get_ylim()[1], alpha=0.3)
ax3.set_xlabel('Total Bill ($)', fontsize=11)
ax3.set_ylabel('Density', fontsize=11)
ax3.set_title('Density Plot (KDE)', fontsize=12, fontweight='bold')
ax3.grid(alpha=0.3)

# 4. Violin Plot
ax4 = fig.add_subplot(gs[1, 2])
parts = ax4.violinplot([tips['total_bill']], vert=True, showmeans=True, showmedians=True)
for pc in parts['bodies']:
    pc.set_facecolor('lightcoral')
    pc.set_alpha(0.7)
ax4.set_ylabel('Total Bill ($)', fontsize=11)
ax4.set_title('Violin Plot', fontsize=12, fontweight='bold')
ax4.set_xticks([1])
ax4.set_xticklabels(['Total Bill'])
ax4.grid(axis='y', alpha=0.3)

# 5. Q-Q Plot
ax5 = fig.add_subplot(gs[2, :2])
stats.probplot(tips['total_bill'], dist="norm", plot=ax5)
ax5.set_title('Q-Q Plot (Normality Check)', fontsize=12, fontweight='bold')
ax5.grid(alpha=0.3)

# 6. Cumulative Distribution
ax6 = fig.add_subplot(gs[2, 2])
sorted_data = np.sort(tips['total_bill'])
cumulative = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
ax6.plot(sorted_data, cumulative, linewidth=2)
ax6.set_xlabel('Total Bill ($)', fontsize=11)
ax6.set_ylabel('Cumulative Probability', fontsize=11)
ax6.set_title('Cumulative Distribution', fontsize=12, fontweight='bold')
ax6.grid(alpha=0.3)

plt.suptitle('Comprehensive Numerical Variable Analysis', fontsize=14, fontweight='bold', y=0.995)
plt.show()

In [ ]:
# Compare multiple numerical variables
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

variables = ['total_bill', 'tip']
colors = ['steelblue', 'coral']

for idx, (var, color) in enumerate(zip(variables, colors)):
    # Histogram
    axes[0, idx].hist(tips[var], bins=20, edgecolor='black', alpha=0.7, color=color)
    axes[0, idx].set_xlabel(var.replace('_', ' ').title())
    axes[0, idx].set_ylabel('Frequency')
    axes[0, idx].set_title(f'{var.replace("_", " ").title()} Distribution', fontweight='bold')
    axes[0, idx].grid(axis='y', alpha=0.3)
    
    # Statistics text
    mean_val = tips[var].mean()
    median_val = tips[var].median()
    std_val = tips[var].std()
    skew_val = skew(tips[var])
    
    stats_text = f'Mean: ${mean_val:.2f}\nMedian: ${median_val:.2f}\nStd: ${std_val:.2f}\nSkew: {skew_val:.2f}'
    axes[0, idx].text(0.7, 0.95, stats_text, transform=axes[0, idx].transAxes,
                      verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Box plot
    bp = axes[1, idx].boxplot(tips[var], vert=True, patch_artist=True,
                               boxprops=dict(facecolor=color, alpha=0.7))
    axes[1, idx].set_ylabel(var.replace('_', ' ').title())
    axes[1, idx].set_title(f'{var.replace("_", " ").title()} Box Plot', fontweight='bold')
    axes[1, idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Analyzing Categorical Variables <a id='analyzing-categorical'></a>

Now let's analyze categorical variables.

In [ ]:
# Analyze 'day' from tips dataset
print("CATEGORICAL VARIABLE ANALYSIS: Day")
print("="*80)

# Frequency table
day_counts = tips['day'].value_counts()
day_pct = tips['day'].value_counts(normalize=True) * 100

freq_table = pd.DataFrame({
    'Count': day_counts,
    'Percentage': day_pct,
    'Cumulative %': day_pct.cumsum()
})

print("\nFrequency Table:")
print(freq_table.round(2))

# Basic statistics
print(f"\nTotal observations: {tips['day'].count()}")
print(f"Missing values: {tips['day'].isnull().sum()}")
print(f"Unique categories: {tips['day'].nunique()}")
print(f"Mode (most common): {tips['day'].mode().values[0]}")
print(f"Mode frequency: {day_counts.iloc[0]} ({day_pct.iloc[0]:.1f}%)")

In [ ]:
# Multiple categorical variables
print("\nCATEGORICAL VARIABLES COMPARISON")
print("="*80)

categorical_vars = ['day', 'time', 'sex', 'smoker']

for var in categorical_vars:
    print(f"\n{var.upper()}:")
    print(f"  Unique values: {tips[var].nunique()}")
    print(f"  Mode: {tips[var].mode().values[0]}")
    print(f"  Top 3 categories:")
    
    top3 = tips[var].value_counts().head(3)
    for category, count in top3.items():
        pct = count / len(tips) * 100
        print(f"    {category}: {count} ({pct:.1f}%)")

## 9. Visualizing Categorical Variables <a id='visualizing-categorical'></a>

Create effective visualizations for categorical data.

In [ ]:
# Comprehensive categorical visualizations
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Bar Chart (Day)
day_counts = tips['day'].value_counts()
axes[0, 0].bar(day_counts.index, day_counts.values, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('Day')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Bar Chart: Day', fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Horizontal Bar Chart (sorted)
sorted_counts = tips['day'].value_counts().sort_values()
axes[0, 1].barh(sorted_counts.index, sorted_counts.values, edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].set_xlabel('Count')
axes[0, 1].set_ylabel('Day')
axes[0, 1].set_title('Horizontal Bar Chart (Sorted)', fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Pie Chart (Time)
time_counts = tips['time'].value_counts()
colors = ['#ff9999', '#66b3ff']
axes[0, 2].pie(time_counts, labels=time_counts.index, autopct='%1.1f%%', 
               startangle=90, colors=colors)
axes[0, 2].set_title('Pie Chart: Time', fontweight='bold')

# 4. Count Plot with Seaborn (Sex)
sns.countplot(data=tips, x='sex', ax=axes[1, 0], palette='Set2')
axes[1, 0].set_xlabel('Sex')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Count Plot: Sex', fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# 5. Grouped Count Plot (Smoker by Day)
sns.countplot(data=tips, x='day', hue='smoker', ax=axes[1, 1], palette='Set1')
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Grouped Count Plot: Smoker by Day', fontweight='bold')
axes[1, 1].legend(title='Smoker')
axes[1, 1].grid(axis='y', alpha=0.3)

# 6. Percentage Stacked Bar (Smoker by Time)
smoker_time = pd.crosstab(tips['time'], tips['smoker'], normalize='index') * 100
smoker_time.plot(kind='bar', stacked=True, ax=axes[1, 2], 
                 color=['#90EE90', '#FFB6C1'], alpha=0.8)
axes[1, 2].set_xlabel('Time')
axes[1, 2].set_ylabel('Percentage')
axes[1, 2].set_title('Stacked Bar: Smoker by Time (%)', fontweight='bold')
axes[1, 2].legend(title='Smoker')
axes[1, 2].tick_params(axis='x', rotation=0)
axes[1, 2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Advanced categorical visualization
print("ADVANCED CATEGORICAL ANALYSIS")
print("="*80)

# Create a more detailed analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Bar chart with value labels
day_counts = tips['day'].value_counts()
bars = axes[0, 0].bar(day_counts.index, day_counts.values, 
                       edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 0].set_xlabel('Day', fontsize=11)
axes[0, 0].set_ylabel('Count', fontsize=11)
axes[0, 0].set_title('Day Distribution with Labels', fontweight='bold', fontsize=12)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                     f'{int(height)}',
                     ha='center', va='bottom', fontsize=10)
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Sorted bar with percentages
day_pct = tips['day'].value_counts(normalize=True).sort_values(ascending=True) * 100
bars2 = axes[0, 1].barh(day_pct.index, day_pct.values, 
                         edgecolor='black', alpha=0.7, color='lightcoral')
axes[0, 1].set_xlabel('Percentage (%)', fontsize=11)
axes[0, 1].set_ylabel('Day', fontsize=11)
axes[0, 1].set_title('Day Distribution (Percentage)', fontweight='bold', fontsize=12)

for bar in bars2:
    width = bar.get_width()
    axes[0, 1].text(width, bar.get_y() + bar.get_height()/2.,
                     f'{width:.1f}%',
                     ha='left', va='center', fontsize=10)
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Multiple categories comparison
cat_vars = ['day', 'time', 'sex', 'smoker']
cardinalities = [tips[var].nunique() for var in cat_vars]

bars3 = axes[1, 0].bar(cat_vars, cardinalities, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1, 0].set_xlabel('Variable', fontsize=11)
axes[1, 0].set_ylabel('Number of Unique Categories', fontsize=11)
axes[1, 0].set_title('Cardinality Comparison', fontweight='bold', fontsize=12)
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

for bar in bars3:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                     f'{int(height)}',
                     ha='center', va='bottom', fontsize=10)

# 4. Category proportions table as heatmap
cat_proportions = []
for var in cat_vars:
    counts = tips[var].value_counts()
    cat_proportions.append(counts.values[:3] if len(counts) >= 3 else list(counts.values) + [0]*(3-len(counts)))

cat_proportions_df = pd.DataFrame(cat_proportions, 
                                   index=cat_vars, 
                                   columns=['Top 1', 'Top 2', 'Top 3'])

sns.heatmap(cat_proportions_df, annot=True, fmt='d', cmap='YlOrRd', 
            ax=axes[1, 1], cbar_kws={'label': 'Count'})
axes[1, 1].set_title('Top 3 Category Frequencies', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Rank', fontsize=11)
axes[1, 1].set_ylabel('Variable', fontsize=11)

plt.tight_layout()
plt.show()

## 10. Distribution Analysis <a id='distribution-analysis'></a>

Test for normality and analyze distributions.

In [ ]:
# Normality testing
print("NORMALITY ANALYSIS: Total Bill")
print("="*80)

# Visual test: Q-Q plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Q-Q plot
stats.probplot(tips['total_bill'], dist="norm", plot=axes[0])
axes[0].set_title('Q-Q Plot for Total Bill', fontweight='bold')
axes[0].grid(alpha=0.3)

# Histogram with normal curve overlay
axes[1].hist(tips['total_bill'], bins=30, density=True, 
             edgecolor='black', alpha=0.7, color='steelblue', label='Data')

# Overlay normal distribution
mu, sigma = tips['total_bill'].mean(), tips['total_bill'].std()
x = np.linspace(tips['total_bill'].min(), tips['total_bill'].max(), 100)
axes[1].plot(x, norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal Curve')

axes[1].set_xlabel('Total Bill ($)')
axes[1].set_ylabel('Density')
axes[1].set_title('Histogram with Normal Curve Overlay', fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical tests
print("\nSTATISTICAL NORMALITY TESTS")
print("-"*80)

# Shapiro-Wilk test
stat_shapiro, p_shapiro = shapiro(tips['total_bill'])
print(f"Shapiro-Wilk Test:")
print(f"  Statistic: {stat_shapiro:.4f}")
print(f"  p-value: {p_shapiro:.4f}")

if p_shapiro > 0.05:
    print("  ✅ Appears normally distributed (fail to reject H0)")
else:
    print("  ❌ NOT normally distributed (reject H0)")

# Kolmogorov-Smirnov test
from scipy.stats import kstest
stat_ks, p_ks = kstest(tips['total_bill'], 'norm', 
                        args=(tips['total_bill'].mean(), tips['total_bill'].std()))
print(f"\nKolmogorov-Smirnov Test:")
print(f"  Statistic: {stat_ks:.4f}")
print(f"  p-value: {p_ks:.4f}")

if p_ks > 0.05:
    print("  ✅ Appears normally distributed")
else:
    print("  ❌ NOT normally distributed")

# Anderson-Darling test
from scipy.stats import anderson
result_anderson = anderson(tips['total_bill'])
print(f"\nAnderson-Darling Test:")
print(f"  Statistic: {result_anderson.statistic:.4f}")
print(f"  Critical values: {result_anderson.critical_values}")
print(f"  Significance levels: {result_anderson.significance_level}")

if result_anderson.statistic < result_anderson.critical_values[2]:  # 5% significance
    print("  ✅ Appears normally distributed at 5% significance")
else:
    print("  ❌ NOT normally distributed at 5% significance")

In [ ]:
# Compare different distributions
print("\nCOMPARING DISTRIBUTIONS")
print("="*80)

# Generate different distribution types
np.random.seed(42)
normal_dist = np.random.normal(50, 10, 1000)
right_skewed = np.random.exponential(20, 1000)
left_skewed = 100 - np.random.exponential(20, 1000)
uniform_dist = np.random.uniform(0, 100, 1000)

distributions = {
    'Normal': normal_dist,
    'Right-Skewed': right_skewed,
    'Left-Skewed': left_skewed,
    'Uniform': uniform_dist
}

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, (name, data) in enumerate(distributions.items()):
    row = idx // 2
    col = (idx % 2) * 2
    
    # Histogram
    axes[row, col].hist(data, bins=30, edgecolor='black', alpha=0.7)
    axes[row, col].set_title(f'{name} Distribution', fontweight='bold')
    axes[row, col].set_xlabel('Value')
    axes[row, col].set_ylabel('Frequency')
    axes[row, col].grid(axis='y', alpha=0.3)
    
    # Add statistics
    stats_text = f'Mean: {data.mean():.2f}\nMedian: {np.median(data):.2f}\nSkew: {skew(data):.2f}\nKurt: {kurtosis(data):.2f}'
    axes[row, col].text(0.7, 0.95, stats_text, transform=axes[row, col].transAxes,
                        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Q-Q plot
    stats.probplot(data, dist="norm", plot=axes[row, col+1])
    axes[row, col+1].set_title(f'{name} Q-Q Plot', fontweight='bold')
    axes[row, col+1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Shapiro-Wilk test for each
print("\nShapiro-Wilk Test Results:")
print("-"*80)
for name, data in distributions.items():
    stat, p = shapiro(data)
    result = "Normal" if p > 0.05 else "Not Normal"
    print(f"{name:15s}: p-value = {p:.4f} → {result}")

## 11. Complete Analysis Workflows <a id='complete-workflows'></a>

Put it all together in automated workflows.

In [ ]:
def comprehensive_numerical_analysis(data, var_name):
    """
    Complete univariate analysis for numerical variables.
    """
    print("="*80)
    print(f"COMPREHENSIVE NUMERICAL ANALYSIS: {var_name}")
    print("="*80)
    
    # 1. Descriptive statistics
    print("\n1. DESCRIPTIVE STATISTICS")
    print("-"*80)
    print(data.describe())
    
    print(f"\nCentral Tendency:")
    print(f"  Mean:   {data.mean():.2f}")
    print(f"  Median: {data.median():.2f}")
    
    print(f"\nDispersion:")
    print(f"  Std Dev: {data.std():.2f}")
    print(f"  IQR: {data.quantile(0.75) - data.quantile(0.25):.2f}")
    print(f"  Range: [{data.min():.2f}, {data.max():.2f}]")
    
    print(f"\nShape:")
    print(f"  Skewness: {skew(data.dropna()):.3f}")
    print(f"  Kurtosis: {kurtosis(data.dropna()):.3f}")
    
    # 2. Normality test
    print("\n2. NORMALITY TEST")
    print("-"*80)
    stat, p = shapiro(data.dropna())
    print(f"Shapiro-Wilk p-value: {p:.4f}")
    print(f"Result: {'Normal' if p > 0.05 else 'Not Normal'}")
    
    # 3. Visualizations
    print("\n3. GENERATING VISUALIZATIONS...")
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    
    # Histogram
    axes[0, 0].hist(data.dropna(), bins=30, edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean')
    axes[0, 0].axvline(data.median(), color='green', linestyle='--', linewidth=2, label=f'Median')
    axes[0, 0].set_xlabel(var_name)
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Histogram')
    axes[0, 0].legend()
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # Box plot
    bp = axes[0, 1].boxplot(data.dropna(), vert=True, patch_artist=True)
    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')
    axes[0, 1].set_ylabel(var_name)
    axes[0, 1].set_title('Box Plot')
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # Density plot
    data.plot(kind='density', ax=axes[0, 2], linewidth=2)
    axes[0, 2].set_xlabel(var_name)
    axes[0, 2].set_ylabel('Density')
    axes[0, 2].set_title('Density Plot')
    axes[0, 2].grid(alpha=0.3)
    
    # Q-Q plot
    stats.probplot(data.dropna(), dist="norm", plot=axes[1, 0])
    axes[1, 0].set_title('Q-Q Plot')
    axes[1, 0].grid(alpha=0.3)
    
    # Violin plot
    parts = axes[1, 1].violinplot([data.dropna()], vert=True, showmeans=True, showmedians=True)
    axes[1, 1].set_ylabel(var_name)
    axes[1, 1].set_title('Violin Plot')
    axes[1, 1].grid(axis='y', alpha=0.3)
    
    # Cumulative distribution
    sorted_data = np.sort(data.dropna())
    cumulative = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    axes[1, 2].plot(sorted_data, cumulative, linewidth=2)
    axes[1, 2].set_xlabel(var_name)
    axes[1, 2].set_ylabel('Cumulative Probability')
    axes[1, 2].set_title('Cumulative Distribution')
    axes[1, 2].grid(alpha=0.3)
    
    plt.suptitle(f'Comprehensive Analysis: {var_name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)
    print("✅ ANALYSIS COMPLETE")
    print("="*80)

# Example usage
comprehensive_numerical_analysis(tips['total_bill'], 'Total Bill ($)')

In [ ]:
def comprehensive_categorical_analysis(data, var_name):
    """
    Complete univariate analysis for categorical variables.
    """
    print("="*80)
    print(f"COMPREHENSIVE CATEGORICAL ANALYSIS: {var_name}")
    print("="*80)
    
    # 1. Basic statistics
    print("\n1. BASIC STATISTICS")
    print("-"*80)
    print(f"Count: {data.count()}")
    print(f"Missing: {data.isnull().sum()} ({data.isnull().mean()*100:.1f}%)")
    print(f"Unique categories: {data.nunique()}")
    print(f"Mode: {data.mode().values[0] if len(data.mode()) > 0 else 'N/A'}")
    
    # 2. Frequency table
    print("\n2. FREQUENCY TABLE")
    print("-"*80)
    freq_table = pd.DataFrame({
        'Count': data.value_counts(),
        'Percentage': data.value_counts(normalize=True) * 100
    })
    freq_table['Cumulative %'] = freq_table['Percentage'].cumsum()
    print(freq_table.round(2))
    
    # 3. Visualizations
    print("\n3. GENERATING VISUALIZATIONS...")
    
    num_categories = data.nunique()
    
    if num_categories <= 5:
        fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    else:
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Bar chart
    counts = data.value_counts()
    ax_idx = 0
    axes[ax_idx].bar(counts.index, counts.values, edgecolor='black', alpha=0.7)
    axes[ax_idx].set_xlabel(var_name)
    axes[ax_idx].set_ylabel('Count')
    axes[ax_idx].set_title(f'{var_name} Distribution (Bar Chart)')
    axes[ax_idx].tick_params(axis='x', rotation=45)
    axes[ax_idx].grid(axis='y', alpha=0.3)
    
    # Horizontal bar (sorted)
    ax_idx += 1
    sorted_counts = counts.sort_values()
    axes[ax_idx].barh(sorted_counts.index, sorted_counts.values, edgecolor='black', alpha=0.7)
    axes[ax_idx].set_xlabel('Count')
    axes[ax_idx].set_ylabel(var_name)
    axes[ax_idx].set_title(f'{var_name} Distribution (Sorted)')
    axes[ax_idx].grid(axis='x', alpha=0.3)
    
    # Pie chart (only if <= 5 categories)
    if num_categories <= 5:
        ax_idx += 1
        axes[ax_idx].pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
        axes[ax_idx].set_title(f'{var_name} Distribution (Pie Chart)')
    
    plt.suptitle(f'Categorical Analysis: {var_name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)
    print("✅ ANALYSIS COMPLETE")
    print("="*80)

# Example usage
comprehensive_categorical_analysis(tips['day'], 'Day of Week')

In [ ]:
# Analyze all variables in a dataset
print("COMPLETE DATASET ANALYSIS: Tips Dataset")
print("="*80)

# Identify variable types
numerical_vars = tips.select_dtypes(include=[np.number]).columns.tolist()
categorical_vars = tips.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\nNumerical variables ({len(numerical_vars)}): {numerical_vars}")
print(f"Categorical variables ({len(categorical_vars)}): {categorical_vars}")

# Quick summary for all numerical variables
print("\n" + "="*80)
print("NUMERICAL VARIABLES SUMMARY")
print("="*80)

summary_stats = []
for var in numerical_vars:
    summary_stats.append({
        'Variable': var,
        'Mean': tips[var].mean(),
        'Median': tips[var].median(),
        'Std': tips[var].std(),
        'Skewness': skew(tips[var]),
        'Kurtosis': kurtosis(tips[var])
    })

summary_df = pd.DataFrame(summary_stats).round(3)
print(summary_df.to_string(index=False))

# Quick summary for all categorical variables
print("\n" + "="*80)
print("CATEGORICAL VARIABLES SUMMARY")
print("="*80)

for var in categorical_vars:
    print(f"\n{var.upper()}:")
    print(f"  Unique: {tips[var].nunique()}")
    print(f"  Mode: {tips[var].mode().values[0]}")
    print(f"  Top 3: {list(tips[var].value_counts().head(3).index)}")

print("\n" + "="*80)
print("✅ DATASET ANALYSIS COMPLETE")
print("="*80)

## Summary

**What We Accomplished:**

1. ✅ Calculated central tendency (mean, median, mode)
2. ✅ Measured dispersion (std, IQR, range, variance)
3. ✅ Analyzed distribution shape (skewness, kurtosis)
4. ✅ Created comprehensive visualizations (histogram, box plot, density, violin, Q-Q)
5. ✅ Analyzed categorical variables (frequency, percentages)
6. ✅ Created categorical visualizations (bar, pie, count plots)
7. ✅ Tested for normality (Shapiro-Wilk, K-S, Anderson-Darling)
8. ✅ Built automated analysis workflows

**Key Takeaways:**

- 🎯 **Multiple measures needed** - Use mean AND median, std AND IQR
- 🎯 **Visualize always** - Charts reveal patterns statistics can't
- 🎯 **Context matters** - Interpret results based on domain
- 🎯 **Check assumptions** - Test normality before using parametric methods
- 🎯 **Automated workflows** - Build reusable analysis functions

**Next Steps:**

- Practice with different datasets
- Complete the exercises notebook
- Move to Unit 6: Outlier Detection and Treatment

# Unit 5: Univariate Analysis - Practical Exercises

## Table of Contents
1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Analyzing Numerical Variables](#analyzing-numerical)
4. [Measures of Central Tendency](#central-tendency)
5. [Measures of Dispersion](#dispersion)
6. [Measures of Shape](#shape)
7. [Visualizing Numerical Variables](#visualizing-numerical)
8. [Analyzing Categorical Variables](#analyzing-categorical)
9. [Visualizing Categorical Variables](#visualizing-categorical)
10. [Distribution Analysis](#distribution-analysis)
11. [Complete Analysis Workflows](#complete-workflows)

---